In [1]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

In [9]:
# Load FinBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertModel.from_pretrained('yiyanghkust/finbert-tone')

In [10]:
df_sdg = pd.read_excel('C:/Users/Buket Fildisi/Desktop/BCU/Dissertation/PHD/SDGTargetsToDo.xlsx')
# topic modelling results
#df_bertarticle= pd.read_excel(r'C:\Users\Buket Fildisi\Desktop\BCU\Dissertation\CodeForDissertation\bertopicResult\Result_opt_umap_ng_BertArticle_paragraph.xlsx')
df_bertLinkedIn= pd.read_excel(r'C:\Users\Buket Fildisi\Desktop\BCU\Last Results_PHD\bert\Result_opt_tfidf_umap_ng_BertArticle_paragraph.xlsx')

In [11]:
df_bertLinkedIn

,Unnamed: 0,Topic,Count,Name,Representation,Representative_Docs
0,0,-1,55,-1_emerging market_balance sheets_financial st...,"['emerging market', 'balance sheets', 'financi...",['global financial stability report: financial...
1,1,0,30,0_balance sheets_financial stability_euro area...,"['balance sheets', 'financial stability', 'eur...",['xii international monetary fund | april 2015...
2,2,1,29,1_directors noted_directors welcomed_emerging ...,"['directors noted', 'directors welcomed', 'eme...","['higher long-term yields, including capital l..."
3,3,2,26,2_financial conditions_global financial_financ...,"['financial conditions', 'global financial', '...",[' international monetary fund | october 2018 ...
4,4,3,14,3_financial stability_global financial_market ...,"['financial stability', 'global financial', 'm...",['global financial stability report: moving fr...
5,5,4,10,4_imf staff_imf staff calculations_staff calcu...,"['imf staff', 'imf staff calculations', 'staff...",['executive summary international monetary fu...


In [12]:
import string
import re
def text_clean_1(text):
    text = text.lower()
    text= text.replace("'","");
    #text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text
clean_bertlinkedin=[]
#clean_bertarticle=[]
for i in range(len(df_bertLinkedIn)):
    #type(df_bertLinkedIn['Representation'])
    type(df_bertLinkedIn['Name'])
    #clean_bertlinkedin.append(text_clean_1(df_bertLinkedIn['Representation'][i]))
    clean_bertlinkedin.append(text_clean_1(df_bertLinkedIn['Name'][i]))
    print(clean_bertlinkedin)

['-1_emerging market_balance sheets_financial stability_central banks']
['-1_emerging market_balance sheets_financial stability_central banks', '0_balance sheets_financial stability_euro area_financial institutions']
['-1_emerging market_balance sheets_financial stability_central banks', '0_balance sheets_financial stability_euro area_financial institutions', '1_directors noted_directors welcomed_emerging market_market countries']
['-1_emerging market_balance sheets_financial stability_central banks', '0_balance sheets_financial stability_euro area_financial institutions', '1_directors noted_directors welcomed_emerging market_market countries', '2_financial conditions_global financial_financial stability_advanced economies']
['-1_emerging market_balance sheets_financial stability_central banks', '0_balance sheets_financial stability_euro area_financial institutions', '1_directors noted_directors welcomed_emerging market_market countries', '2_financial conditions_global financial_financ

In [13]:
clean_sdg=[]
for i in range(len(df_sdg)):
    type(df_sdg['ThingsToDo'])
    clean_sdg.append(text_clean_1(df_sdg['ThingsToDo'][i]))
    print(clean_sdg)

['find a goal 1 charity you want to support. any donation, big or small, can make a difference!\n\ndonate what you don’t use. local charities will give your gently used clothes, books and furniture a new life.\n\nsupport campaigns collecting items for victims of emergencies. donate your clothes, food supplies etc. to support those in need.\n\npoverty still persists in every country. over 8% of the world population lives in extreme poverty. lend your voice to the fight against extreme poverty.']
['find a goal 1 charity you want to support. any donation, big or small, can make a difference!\n\ndonate what you don’t use. local charities will give your gently used clothes, books and furniture a new life.\n\nsupport campaigns collecting items for victims of emergencies. donate your clothes, food supplies etc. to support those in need.\n\npoverty still persists in every country. over 8% of the world population lives in extreme poverty. lend your voice to the fight against extreme poverty.', 

In [14]:
#s_bertarticle= ''.join(map(str,clean_bertarticle))
s_bertlinkedin= ''.join(map(str,clean_bertlinkedin))
s_sdg= ''.join(map(str,clean_sdg))

In [15]:
# Tokenize the text and topic
#inputs_text = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
#inputs_topic = tokenizer(topic, return_tensors='pt', truncation=True, padding=True)
inputs_text=[]
for i in range(len(clean_bertlinkedin)):
    inputs_text.append(tokenizer(clean_bertlinkedin[i], return_tensors='pt', truncation=True, padding=True))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [16]:
len(clean_bertlinkedin)

6

In [17]:
inputs_topic = [] 
for i in range(17):
    inputs_topic.append(tokenizer(clean_sdg[i], return_tensors='pt', truncation=True, padding=True))

In [18]:
# Generate embeddings for text
bertLink_embed=[]
with torch.no_grad():
    for i in range(len(clean_bertlinkedin)):
        outputs_text = model(**inputs_text[i])
        text_embedding = outputs_text.last_hidden_state.mean(dim=1)
        bertLink_embed.append(text_embedding)

In [19]:
# Generate embeddings for topic
sdgEmbeddingResult=[]
with torch.no_grad():
    for i in range(17):
        outputs_topic= model(**inputs_topic[i])
        topic_embedding=outputs_topic.last_hidden_state.mean(dim=1)
        sdgEmbeddingResult.append(topic_embedding)

In [20]:
# Compute cosine similarity
cosine_sim = cosine_similarity(text_embedding, topic_embedding)
print(f"Cosine Similarity: {cosine_sim.item()}")
cos_sim_test=[]
for i in range(17):
    for j in range(len(clean_bertlinkedin)):
        cos_sim_test.append(cosine_similarity(bertLink_embed[j], sdgEmbeddingResult[i]))
print(f"i : {i}")
print(cos_sim_test)

Cosine Similarity: 0.5296785235404968
i : 16
[array([[0.39460945]], dtype=float32), array([[0.3711362]], dtype=float32), array([[0.4424321]], dtype=float32), array([[0.27249724]], dtype=float32), array([[0.22180973]], dtype=float32), array([[0.5630065]], dtype=float32), array([[0.3552865]], dtype=float32), array([[0.3286602]], dtype=float32), array([[0.40548062]], dtype=float32), array([[0.2673452]], dtype=float32), array([[0.2239296]], dtype=float32), array([[0.49968553]], dtype=float32), array([[0.41011235]], dtype=float32), array([[0.40468293]], dtype=float32), array([[0.4743655]], dtype=float32), array([[0.3553382]], dtype=float32), array([[0.31858778]], dtype=float32), array([[0.49483892]], dtype=float32), array([[0.39681336]], dtype=float32), array([[0.36366224]], dtype=float32), array([[0.4743561]], dtype=float32), array([[0.29039884]], dtype=float32), array([[0.2647419]], dtype=float32), array([[0.52607125]], dtype=float32), array([[0.4113372]], dtype=float32), array([[0.387571

In [21]:
import numpy as np
# Convert the list to a NumPy array and reshape it to 49 rows and 17 columns
reshaped_array = np.array(cos_sim_test).reshape(len(clean_bertlinkedin), 17)
# Convert the NumPy array back to a list of lists (optional)
reshaped_list = reshaped_array.tolist()
# Print the reshaped list to verify
for row in reshaped_list:
    print(row)


[0.3946094512939453, 0.3711361885070801, 0.44243210554122925, 0.2724972367286682, 0.2218097299337387, 0.5630065202713013, 0.35528650879859924, 0.32866019010543823, 0.40548062324523926, 0.2673451900482178, 0.22392959892749786, 0.49968552589416504, 0.4101123511791229, 0.4046829342842102, 0.4743655025959015, 0.3553381860256195, 0.3185877799987793]
[0.4948389232158661, 0.39681336283683777, 0.3636622428894043, 0.47435611486434937, 0.29039883613586426, 0.2647418975830078, 0.5260712504386902, 0.4113371968269348, 0.38757118582725525, 0.5036237239837646, 0.3108145594596863, 0.2635681629180908, 0.538044810295105, 0.3872146010398865, 0.3577783703804016, 0.4662094712257385, 0.25520893931388855]
[0.20219752192497253, 0.5511530041694641, 0.36131393909454346, 0.32000797986984253, 0.40297120809555054, 0.22575730085372925, 0.1966816782951355, 0.505139946937561, 0.398537278175354, 0.3899245858192444, 0.503585696220398, 0.3663642406463623, 0.32146528363227844, 0.4751332998275757, 0.4287235140800476, 0.40

In [22]:
max(reshaped_list[1])

0.538044810295105

In [23]:
sdg_columns = [
    'Goal 1_No poverty',
    'Goal 2_ZeroHunger',
    'Goal 3_GoodHealth&WellBeing',
    'Goal 4_QualityEducation',
    'Goal 5_GenderEquality',
    'Goal 6_CleanWater&Sanitation',
    'Goal 7_Affordable&CleanEnergy',
    'Goal 8_DecentWork&EconomicGrowth',
    'Goal 9_IndustryInnovation&Infrastructure',
    'Goal 10_ReduceInequality',
    'Goal 11_SustainableCities&Communities',
    'Goal 12_ResponsibleConsumption&Production',
    'Goal 13_ClimateAction',
    'Goal 14_LifeBelowWater',
    'Goal 15_LifeOnLand',
    'Goal 16_PeaceJustice&StrongInstitutions',
    'Goal 17_PartnershipsForTheGoals'
]

In [24]:
df_Name = pd.DataFrame(reshaped_list, columns=sdg_columns)
df_Name

,Goal 1_No poverty,Goal 2_ZeroHunger,Goal 3_GoodHealth&WellBeing,Goal 4_QualityEducation,Goal 5_GenderEquality,Goal 6_CleanWater&Sanitation,Goal 7_Affordable&CleanEnergy,Goal 8_DecentWork&EconomicGrowth,Goal 9_IndustryInnovation&Infrastructure,Goal 10_ReduceInequality,Goal 11_SustainableCities&Communities,Goal 12_ResponsibleConsumption&Production,Goal 13_ClimateAction,Goal 14_LifeBelowWater,Goal 15_LifeOnLand,Goal 16_PeaceJustice&StrongInstitutions,Goal 17_PartnershipsForTheGoals
0,0.394609,0.371136,0.442432,0.272497,0.221810,0.563007,0.355287,0.328660,0.405481,0.267345,0.223930,0.499686,0.410112,0.404683,0.474366,0.355338,0.318588
1,0.494839,0.396813,0.363662,0.474356,0.290399,0.264742,0.526071,0.411337,0.387571,0.503624,0.310815,0.263568,0.538045,0.387215,0.357778,0.466209,0.255209
2,0.202198,0.551153,0.361314,0.320008,0.402971,0.225757,0.196682,0.505140,0.398537,0.389925,0.503586,0.366364,0.321465,0.475133,0.428724,0.406359,0.512542
3,0.371262,0.349429,0.482598,0.454563,0.425109,0.534554,0.328584,0.259206,0.590367,0.399800,0.361675,0.486456,0.295239,0.263355,0.515325,0.391271,0.359900
4,0.467755,0.293673,0.255948,0.506723,0.390735,0.352880,0.438089,0.278637,0.243558,0.544216,0.363794,0.342353,0.424265,0.261643,0.205441,0.546306,0.431571
5,0.402143,0.471724,0.311624,0.259741,0.582753,0.417811,0.390928,0.531555,0.338279,0.291271,0.468143,0.484779,0.449383,0.586738,0.439754,0.408199,0.529679


In [25]:
df_rep = pd.DataFrame(reshaped_list, columns=sdg_columns)
df_rep

,Goal 1_No poverty,Goal 2_ZeroHunger,Goal 3_GoodHealth&WellBeing,Goal 4_QualityEducation,Goal 5_GenderEquality,Goal 6_CleanWater&Sanitation,Goal 7_Affordable&CleanEnergy,Goal 8_DecentWork&EconomicGrowth,Goal 9_IndustryInnovation&Infrastructure,Goal 10_ReduceInequality,Goal 11_SustainableCities&Communities,Goal 12_ResponsibleConsumption&Production,Goal 13_ClimateAction,Goal 14_LifeBelowWater,Goal 15_LifeOnLand,Goal 16_PeaceJustice&StrongInstitutions,Goal 17_PartnershipsForTheGoals
0,0.394609,0.371136,0.442432,0.272497,0.221810,0.563007,0.355287,0.328660,0.405481,0.267345,0.223930,0.499686,0.410112,0.404683,0.474366,0.355338,0.318588
1,0.494839,0.396813,0.363662,0.474356,0.290399,0.264742,0.526071,0.411337,0.387571,0.503624,0.310815,0.263568,0.538045,0.387215,0.357778,0.466209,0.255209
2,0.202198,0.551153,0.361314,0.320008,0.402971,0.225757,0.196682,0.505140,0.398537,0.389925,0.503586,0.366364,0.321465,0.475133,0.428724,0.406359,0.512542
3,0.371262,0.349429,0.482598,0.454563,0.425109,0.534554,0.328584,0.259206,0.590367,0.399800,0.361675,0.486456,0.295239,0.263355,0.515325,0.391271,0.359900
4,0.467755,0.293673,0.255948,0.506723,0.390735,0.352880,0.438089,0.278637,0.243558,0.544216,0.363794,0.342353,0.424265,0.261643,0.205441,0.546306,0.431571
5,0.402143,0.471724,0.311624,0.259741,0.582753,0.417811,0.390928,0.531555,0.338279,0.291271,0.468143,0.484779,0.449383,0.586738,0.439754,0.408199,0.529679


In [26]:
df_Linkedin=df_Name.copy().reset_index()
df_Linkedin

,index,Goal 1_No poverty,Goal 2_ZeroHunger,Goal 3_GoodHealth&WellBeing,Goal 4_QualityEducation,Goal 5_GenderEquality,Goal 6_CleanWater&Sanitation,Goal 7_Affordable&CleanEnergy,Goal 8_DecentWork&EconomicGrowth,Goal 9_IndustryInnovation&Infrastructure,Goal 10_ReduceInequality,Goal 11_SustainableCities&Communities,Goal 12_ResponsibleConsumption&Production,Goal 13_ClimateAction,Goal 14_LifeBelowWater,Goal 15_LifeOnLand,Goal 16_PeaceJustice&StrongInstitutions,Goal 17_PartnershipsForTheGoals
0,0,0.394609,0.371136,0.442432,0.272497,0.221810,0.563007,0.355287,0.328660,0.405481,0.267345,0.223930,0.499686,0.410112,0.404683,0.474366,0.355338,0.318588
1,1,0.494839,0.396813,0.363662,0.474356,0.290399,0.264742,0.526071,0.411337,0.387571,0.503624,0.310815,0.263568,0.538045,0.387215,0.357778,0.466209,0.255209
2,2,0.202198,0.551153,0.361314,0.320008,0.402971,0.225757,0.196682,0.505140,0.398537,0.389925,0.503586,0.366364,0.321465,0.475133,0.428724,0.406359,0.512542
3,3,0.371262,0.349429,0.482598,0.454563,0.425109,0.534554,0.328584,0.259206,0.590367,0.399800,0.361675,0.486456,0.295239,0.263355,0.515325,0.391271,0.359900
4,4,0.467755,0.293673,0.255948,0.506723,0.390735,0.352880,0.438089,0.278637,0.243558,0.544216,0.363794,0.342353,0.424265,0.261643,0.205441,0.546306,0.431571
5,5,0.402143,0.471724,0.311624,0.259741,0.582753,0.417811,0.390928,0.531555,0.338279,0.291271,0.468143,0.484779,0.449383,0.586738,0.439754,0.408199,0.529679


In [27]:
df_Linkedin_Name=pd.merge(df_bertLinkedIn[['Name']].reset_index(),df_Linkedin, how="left", on=["index"])
df_Linkedin_Name

,index,Name,Goal 1_No poverty,Goal 2_ZeroHunger,Goal 3_GoodHealth&WellBeing,Goal 4_QualityEducation,Goal 5_GenderEquality,Goal 6_CleanWater&Sanitation,Goal 7_Affordable&CleanEnergy,Goal 8_DecentWork&EconomicGrowth,Goal 9_IndustryInnovation&Infrastructure,Goal 10_ReduceInequality,Goal 11_SustainableCities&Communities,Goal 12_ResponsibleConsumption&Production,Goal 13_ClimateAction,Goal 14_LifeBelowWater,Goal 15_LifeOnLand,Goal 16_PeaceJustice&StrongInstitutions,Goal 17_PartnershipsForTheGoals
0,0,-1_emerging market_balance sheets_financial st...,0.394609,0.371136,0.442432,0.272497,0.221810,0.563007,0.355287,0.328660,0.405481,0.267345,0.223930,0.499686,0.410112,0.404683,0.474366,0.355338,0.318588
1,1,0_balance sheets_financial stability_euro area...,0.494839,0.396813,0.363662,0.474356,0.290399,0.264742,0.526071,0.411337,0.387571,0.503624,0.310815,0.263568,0.538045,0.387215,0.357778,0.466209,0.255209
2,2,1_directors noted_directors welcomed_emerging ...,0.202198,0.551153,0.361314,0.320008,0.402971,0.225757,0.196682,0.505140,0.398537,0.389925,0.503586,0.366364,0.321465,0.475133,0.428724,0.406359,0.512542
3,3,2_financial conditions_global financial_financ...,0.371262,0.349429,0.482598,0.454563,0.425109,0.534554,0.328584,0.259206,0.590367,0.399800,0.361675,0.486456,0.295239,0.263355,0.515325,0.391271,0.359900
4,4,3_financial stability_global financial_market ...,0.467755,0.293673,0.255948,0.506723,0.390735,0.352880,0.438089,0.278637,0.243558,0.544216,0.363794,0.342353,0.424265,0.261643,0.205441,0.546306,0.431571
5,5,4_imf staff_imf staff calculations_staff calcu...,0.402143,0.471724,0.311624,0.259741,0.582753,0.417811,0.390928,0.531555,0.338279,0.291271,0.468143,0.484779,0.449383,0.586738,0.439754,0.408199,0.529679


In [28]:
df_Linkedin.set_index('index', inplace=True)
df_Linkedin

,Goal 1_No poverty,Goal 2_ZeroHunger,Goal 3_GoodHealth&WellBeing,Goal 4_QualityEducation,Goal 5_GenderEquality,Goal 6_CleanWater&Sanitation,Goal 7_Affordable&CleanEnergy,Goal 8_DecentWork&EconomicGrowth,Goal 9_IndustryInnovation&Infrastructure,Goal 10_ReduceInequality,Goal 11_SustainableCities&Communities,Goal 12_ResponsibleConsumption&Production,Goal 13_ClimateAction,Goal 14_LifeBelowWater,Goal 15_LifeOnLand,Goal 16_PeaceJustice&StrongInstitutions,Goal 17_PartnershipsForTheGoals
index,,,,,,,,,,,,,,,,,
0,0.394609,0.371136,0.442432,0.272497,0.221810,0.563007,0.355287,0.328660,0.405481,0.267345,0.223930,0.499686,0.410112,0.404683,0.474366,0.355338,0.318588
1,0.494839,0.396813,0.363662,0.474356,0.290399,0.264742,0.526071,0.411337,0.387571,0.503624,0.310815,0.263568,0.538045,0.387215,0.357778,0.466209,0.255209
2,0.202198,0.551153,0.361314,0.320008,0.402971,0.225757,0.196682,0.505140,0.398537,0.389925,0.503586,0.366364,0.321465,0.475133,0.428724,0.406359,0.512542
3,0.371262,0.349429,0.482598,0.454563,0.425109,0.534554,0.328584,0.259206,0.590367,0.399800,0.361675,0.486456,0.295239,0.263355,0.515325,0.391271,0.359900
4,0.467755,0.293673,0.255948,0.506723,0.390735,0.352880,0.438089,0.278637,0.243558,0.544216,0.363794,0.342353,0.424265,0.261643,0.205441,0.546306,0.431571
5,0.402143,0.471724,0.311624,0.259741,0.582753,0.417811,0.390928,0.531555,0.338279,0.291271,0.468143,0.484779,0.449383,0.586738,0.439754,0.408199,0.529679


In [29]:
top_3_columns = df_Linkedin.apply(lambda row: row.nlargest(3).index.tolist(), axis=1)
top_3_columns_df = pd.DataFrame(top_3_columns.tolist(), index=top_3_columns.index)
top_3_columns_df.columns = ['GOAL', 'Top2', 'Top3']
top_3_columns_df

,GOAL,Top2,Top3
index,,,
0,Goal 6_CleanWater&Sanitation,Goal 12_ResponsibleConsumption&Production,Goal 15_LifeOnLand
1,Goal 13_ClimateAction,Goal 7_Affordable&CleanEnergy,Goal 10_ReduceInequality
2,Goal 2_ZeroHunger,Goal 17_PartnershipsForTheGoals,Goal 8_DecentWork&EconomicGrowth
3,Goal 9_IndustryInnovation&Infrastructure,Goal 6_CleanWater&Sanitation,Goal 15_LifeOnLand
4,Goal 16_PeaceJustice&StrongInstitutions,Goal 10_ReduceInequality,Goal 4_QualityEducation
5,Goal 14_LifeBelowWater,Goal 5_GenderEquality,Goal 8_DecentWork&EconomicGrowth


In [30]:
top_3_columns

index
0    [Goal 6_CleanWater&Sanitation, Goal 12_Respons...
1    [Goal 13_ClimateAction, Goal 7_Affordable&Clea...
2    [Goal 2_ZeroHunger, Goal 17_PartnershipsForThe...
3    [Goal 9_IndustryInnovation&Infrastructure, Goa...
4    [Goal 16_PeaceJustice&StrongInstitutions, Goal...
5    [Goal 14_LifeBelowWater, Goal 5_GenderEquality...
dtype: object

In [31]:
x=df_bertLinkedIn[['Name','Representation']].reset_index()
x.head()

,index,Name,Representation
0,0,-1_emerging market_balance sheets_financial st...,"['emerging market', 'balance sheets', 'financi..."
1,1,0_balance sheets_financial stability_euro area...,"['balance sheets', 'financial stability', 'eur..."
2,2,1_directors noted_directors welcomed_emerging ...,"['directors noted', 'directors welcomed', 'eme..."
3,3,2_financial conditions_global financial_financ...,"['financial conditions', 'global financial', '..."
4,4,3_financial stability_global financial_market ...,"['financial stability', 'global financial', 'm..."


In [32]:
df_Linkedin_t3=pd.merge(df_bertLinkedIn[['Name','Representation']].reset_index(),top_3_columns_df, how="left", on=["index"])
df_Linkedin_t3.head()

,index,Name,Representation,GOAL,Top2,Top3
0,0,-1_emerging market_balance sheets_financial st...,"['emerging market', 'balance sheets', 'financi...",Goal 6_CleanWater&Sanitation,Goal 12_ResponsibleConsumption&Production,Goal 15_LifeOnLand
1,1,0_balance sheets_financial stability_euro area...,"['balance sheets', 'financial stability', 'eur...",Goal 13_ClimateAction,Goal 7_Affordable&CleanEnergy,Goal 10_ReduceInequality
2,2,1_directors noted_directors welcomed_emerging ...,"['directors noted', 'directors welcomed', 'eme...",Goal 2_ZeroHunger,Goal 17_PartnershipsForTheGoals,Goal 8_DecentWork&EconomicGrowth
3,3,2_financial conditions_global financial_financ...,"['financial conditions', 'global financial', '...",Goal 9_IndustryInnovation&Infrastructure,Goal 6_CleanWater&Sanitation,Goal 15_LifeOnLand
4,4,3_financial stability_global financial_market ...,"['financial stability', 'global financial', 'm...",Goal 16_PeaceJustice&StrongInstitutions,Goal 10_ReduceInequality,Goal 4_QualityEducation


In [33]:
df_Linkedin_t3sdg=pd.merge(df_Linkedin_t3,df_sdg,how="left",on=["GOAL"])
df_Linkedin_t3sdg

,index,Name,Representation,GOAL,Top2,Top3,ESG_Target,TARGET,ThingsToDo,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10
0,0,-1_emerging market_balance sheets_financial st...,"['emerging market', 'balance sheets', 'financi...",Goal 6_CleanWater&Sanitation,Goal 12_ResponsibleConsumption&Production,Goal 15_LifeOnLand,"Environment, Social",TARGET 6.1\n\nSAFE AND AFFORDABLE DRINKING WAT...,Find a Goal 6 charity you want to support. Any...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0_balance sheets_financial stability_euro area...,"['balance sheets', 'financial stability', 'eur...",Goal 13_ClimateAction,Goal 7_Affordable&CleanEnergy,Goal 10_ReduceInequality,"Environment, Governance",TARGET 13.1\n\nSTRENGTHEN RESILIENCE AND ADAPT...,Find a Goal 13 charity you want to support. An...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1_directors noted_directors welcomed_emerging ...,"['directors noted', 'directors welcomed', 'eme...",Goal 2_ZeroHunger,Goal 17_PartnershipsForTheGoals,Goal 8_DecentWork&EconomicGrowth,Social,UNIVERSAL ACCESS TO SAFE AND NUTRITIOUS FOOD\n...,Find a Goal 2 charity you want to support. Any...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2_financial conditions_global financial_financ...,"['financial conditions', 'global financial', '...",Goal 9_IndustryInnovation&Infrastructure,Goal 6_CleanWater&Sanitation,Goal 15_LifeOnLand,"Environment, Social, Governance","TARGET 9.1\n\nDEVELOP SUSTAINABLE, RESILIENT A...",Find a Goal 9 charity you want to support. Any...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,3_financial stability_global financial_market ...,"['financial stability', 'global financial', 'm...",Goal 16_PeaceJustice&StrongInstitutions,Goal 10_ReduceInequality,Goal 4_QualityEducation,"Social, Governance",TARGET 16.1\n\nREDUCE VIOLENCE EVERYWHERE\nSig...,Support Choose Love and buy real products for ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,4_imf staff_imf staff calculations_staff calcu...,"['imf staff', 'imf staff calculations', 'staff...",Goal 14_LifeBelowWater,Goal 5_GenderEquality,Goal 8_DecentWork&EconomicGrowth,Environment,TARGET 14.1\n\nREDUCE MARINE POLLUTION\nBy 202...,Find a Goal 14 charity you want to support. An...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_Linkedin.to_excel(r'C:\Users\Buket Fildisi\Desktop\BCU\Dissertation\PHD\CosSimResults\Result_FinBert_Linkedin.xlsx')

In [26]:
df_Linkedin.transpose().to_excel(r'C:\Users\Buket Fildisi\Desktop\BCU\Dissertation\PHD\CosSimResults\Result_FinBert_Linkedin_transpose.xlsx')

In [44]:
df_Linkedin_Name.to_excel(r'C:\Users\Buket Fildisi\Desktop\BCU\Dissertation\PHD\CosSimResults\Result_FinBert_Linkedin_Name.xlsx')

In [34]:
df_Linkedin_t3sdg.to_excel(r'C:\Users\Buket Fildisi\Desktop\BCU\Last Results_PHD\Cosine Similarity Results\Result_FinBert_Bertopic_Article_T3SDG.xlsx')

In [108]:
df_sdg.head(2)

,ESG_Target,GOAL,TARGET,ThingsToDo
0,Social,Goal 1_No poverty,Everyone can help to make sure that we meet th...,Find a Goal 1 charity you want to support. Any...
1,Social,Goal 2_ZeroHunger,UNIVERSAL ACCESS TO SAFE AND NUTRITIOUS FOOD\n...,Find a Goal 2 charity you want to support. Any...
